In [1]:
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from IPython.display import display 
from sklearn.metrics import fbeta_score
import model_utils as model_utils
from sklearn.externals import joblib

import json
# Pretty display for notebooks
%matplotlib inline
pd.set_option('display.max_colwidth', -1)


with open('best_config.json') as json_data_file:
    config = json.load(json_data_file)[0]
print(config)
seed = int(config['seed'])
test_size = config['test_size']

dataset = pd.read_csv("datasets/encoded_dataset.csv")
display(dataset.head(n = 5))

labels = dataset[['RIESGO_VIDA']]
features = dataset.drop(['RIESGO_VIDA'], axis = 1)


C:\Users\mrugeles\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


{'seed': 134.0, 'test_size': 0.1, 'impute': 0.0}


,AFEC_DPTO,AFEC_EDADR,AFEC_EDUC,AFEC_GENERO,AFEC_GETNICO,AFEC_MPIO,AFEC_PARENTESCO,AFEC_POBESPECIAL,AFEC_REGAFILIACION,AFEC_TIPOPER,...,PET_COD_DEPTO,PET_MPIO,PET_TIPOPER,PQR_CANAL,PQR_CLASE_SNS,PQR_TIPOATENCION,PQR_TIPOPETICION,TRIM,SERVICIO_NEGADO,RIESGO_VIDA
0,0.413632,0.457947,0.404291,0.422006,0.43569,0.411426,0.498799,0.425020,0.488333,0.436371,...,0.416195,0.420014,0.271184,0.168226,0.457652,0.266635,0.456139,0.4024,0.460713,0
1,0.367341,0.457947,0.447575,0.455866,0.43569,0.371084,0.459542,0.425020,0.420733,0.436371,...,0.375220,0.366720,0.433927,0.281359,0.457652,0.459141,0.456139,0.4024,0.460713,0
2,0.557791,0.457947,0.488072,0.455866,0.43569,0.564257,0.532725,0.425020,0.420733,0.436371,...,0.549517,0.558897,0.433927,0.616036,0.457652,0.459141,0.456139,0.4024,0.460713,0
3,0.409106,0.457947,0.404291,0.455866,0.43569,0.445723,0.532725,0.526102,0.420733,0.436371,...,0.407647,0.447220,0.433927,0.616036,0.457652,0.459141,0.456139,0.4024,0.460713,1
4,0.413632,0.457947,0.404291,0.455866,0.43569,0.411426,0.498799,0.425020,0.488333,0.436371,...,0.416195,0.420014,0.271184,0.168226,0.457652,0.266635,0.456139,0.4024,0.460713,0


In [2]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Split the 'features' and 'labels' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = test_size, random_state = seed, stratify=labels)

# Show the results of the split
print "features_final set has {} samples.".format(features.shape[0])
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])


SyntaxError: invalid syntax (<ipython-input-2-37554305b789>, line 8)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

# Initialize the three models
clf_A = GaussianNB()
clf_B = RandomForestClassifier(random_state = seed)
clf_C = AdaBoostClassifier(random_state = seed)


# Collect results on the learners
dfResults = pd.DataFrame(columns=['learner', 'train_time', 'pred_time', 'f_test', 'f_train'])

for k, clf in enumerate([clf_A, clf_B, clf_C]):
    clf_name = clf.__class__.__name__  
    clf, dfResults = model_utils.train_predict(clf, 1, X_train, y_train, X_test, y_test, dfResults)


In [ ]:
display( dfResults.sort_values(by=['f_test'], ascending = False)[['learner', 'f_test']])


# Tuning Models

## Tuning RandomForestClassifier

In [ ]:
from sklearn.metrics import make_scorer

rf_classifier = RandomForestClassifier(random_state = seed)

rfParameters = {    
  'criterion':['gini', 'entropy'],
  'max_depth':[5, 10, 15, 30],
  'max_features':['auto', 'sqrt', 'log2', None],
  'class_weight': ['balanced', 'balanced_subsample'], 
}

rf_classifier, default_rf_score, tuned_rf_score, cnf_rf_matrix = model_utils.tune_classifier(rf_classifier, rfParameters, X_train, X_test, y_train, y_test)

model_utils.plot_confusion_matrix(cnf_rf_matrix, classes=['Life not as risk', 'Life at risk'], normalize = True)
print("Unoptimized model\n------")
print("F-score on testing data: {:.4f}".format(default_rf_score))
print("\nOptimized Model\n------")
print("Final F-score on the testing data: {:.4f}".format(tuned_rf_score))
joblib.dump(rf_classifier, 'rf_classifier.joblib'))


## Tuning AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada_classifier = AdaBoostClassifier(random_state = seed)

adaParameters = {
  'learning_rate':[0.1, 0.5, 1],
  'algorithm' :['SAMME', 'SAMME.R']
}

ada_classifier, default_ada_score, tuned_ada_score, cnf_ada_matrix = model_utils.tune_classifier(ada_classifier, adaParameters,  X_train, X_test, y_train, y_test)

model_utils.plot_confusion_matrix(cnf_ada_matrix, classes=['Life not as risk', 'Life at risk'], normalize = True)
print("Unoptimized model\n------")
print("F-score on testing data: {:.4f}".format(default_ada_score))
print("\nOptimized Model\n------")
print("Final F-score on the testing data: {:.4f}".format(tuned_ada_score))

joblib.dump(ada_classifier, 'ada_classifier.joblib') 


## Tuning GaussianClassifier


In [ ]:
gauss_classifier = GaussianNB()

gaussParameters = {
  'priors':[None],
  'var_smoothing' :[1e-09]
}

gauss_classifier, default_gauss_score, tuned_gauss_score, cnf_gauss_matrix = model_utils.tune_classifier(gauss_classifier, gaussParameters,  X_train, X_test, y_train, y_test)

model_utils.plot_confusion_matrix(cnf_gauss_matrix, classes=['Life not as risk', 'Life at risk'], normalize = True)
print("Unoptimized model\n------"))
print("F-score on testing data: {:.4f}".format(default_gauss_score))
print("\nOptimized Model\n------")
print("Final F-score on the testing data: {:.4f}".format(tuned_gauss_score))

joblib.dump(gauss_classifier, 'gauss_classifier.joblib') 


# Stacking
## Stacking with to best classifiers

In [ ]:
sclf_two, sclf_score = model_utils.get_stack_two(rf_classifier, ada_classifier, X_train, X_test, y_train, y_test, seed)

joblib.dump(sclf_two, 'sclf_two.joblib') 

model_utils.model_validation('sclf_two.joblib', X_test, y_test)

## Stacking with all classifiers

In [ ]:
sclf_all, sclf_all_score = model_utils.get_stack_all(rf_classifier, ada_classifier, gauss_classifier, X_train, X_test, y_train, y_test, seed)

joblib.dump(sclf_all, 'sclf_all.joblib') 

model_utils.model_validation('sclf_all.joblib', X_test, y_test)